# 0. Try approaches to find an effective way to merge datasets

In [2]:
import pandas as pd
#TN_01
tn01_col = pd.read_csv('TN_01_merged_data_summary.csv', usecols=[0, 1])
print(tn01_col.info())

#TN_16
tn16_col = pd.read_csv('TN_16_merged_data_summary.csv', usecols=[0, 1])
#rename the column 'Unnamed: 0'
tn16_col.rename(columns={'Unnamed: 0': 'header'}, inplace=True)
print(tn16_col.info())

#TN_19
tn19_col = pd.read_csv('TN_19_merged_data_summary.csv', usecols=[0, 1])
tn19_col.rename(columns={'Unnamed: 0': 'header'}, inplace=True)
print(tn19_col.info())

#TN_20
tn20_col = pd.read_csv('TN_20_merged_data_summary.csv', usecols=[0, 1])
tn20_col.rename(columns={'Unnamed: 0': 'header'}, inplace=True)
print(tn20_col.info())

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1578 entries, 0 to 1577
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   header              1578 non-null   object 
 1   Missing Percentage  1578 non-null   float64
dtypes: float64(1), object(1)
memory usage: 24.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   header              262 non-null    object 
 1   Missing Percentage  262 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   header              875 non-null    object 
 1   Missing Percentage  875 non

In [3]:
# filter out columns with missing values larger than 99%
#TN_01
filtered_tn_01_col_99 = tn01_col[tn01_col['Missing Percentage'] < 99]
# Extract the values from the first column and convert to list
tn_01_col_99 = filtered_tn_01_col_99['header'].tolist()
print('Number of columns in TN_01 with missing values less than 99%: ', len(tn_01_col_99))

#TN_16
filtered_tn_16_col_99 = tn16_col[tn16_col['Missing Percentage'] < 99]
tn_16_col_99 = filtered_tn_16_col_99['header'].tolist()
print('Number of columns in TN_16 with missing values less than 99%: ', len(tn_16_col_99))

#TN_19
filtered_tn_19_col_99 = tn19_col[tn19_col['Missing Percentage'] < 99]
tn_19_col_99 = filtered_tn_19_col_99['header'].tolist()
print('Number of columns in TN_19 with missing values less than 99%: ', len(tn_19_col_99))

#TN_20
filtered_tn_20_col_99 = tn20_col[tn20_col['Missing Percentage'] < 99]
tn_20_col_99 = filtered_tn_20_col_99['header'].tolist()
print('Number of columns in TN_20 with missing values less than 99%: ', len(tn_20_col_99))

Number of columns in TN_01 with missing values less than 99%:  296
Number of columns in TN_16 with missing values less than 99%:  205
Number of columns in TN_19 with missing values less than 99%:  295
Number of columns in TN_20 with missing values less than 99%:  652


In [4]:
import dask.dataframe as dd
df_tn01_99 = dd.read_csv('TN_01_Merged_Data.csv', usecols=tn_01_col_99, dtype=str, assume_missing=True)
df_tn01_99.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 296 entries, MaskID to LabID
dtypes: string(296)

In [5]:
tn19_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   header              875 non-null    object 
 1   Missing Percentage  875 non-null    float64
dtypes: float64(1), object(1)
memory usage: 13.8+ KB


In [5]:
df_tn16_99 = dd.read_csv('TN_16_merged_data.csv', usecols=tn_16_col_99, dtype=str, assume_missing=True)
df_tn16_99.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 205 entries, MaskID to LabID
dtypes: string(205)

In [13]:
import dask.dataframe as dd
df_tn19_99 = dd.read_csv('TN_19_merged_data.csv', usecols=tn_19_col_99, dtype=str, assume_missing=True)
df_tn19_99.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 295 entries, MaskID to ExperiencedAnyHypoglycemicEven
dtypes: string(295)

In [7]:
df_tn20_99 = dd.read_csv('TN_20_merged_data.csv', usecols=tn_20_col_99, dtype=str, assume_missing=True)
df_tn20_99.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 652 entries, MaskID to howmanydoseshastheparticipantm
dtypes: string(652)

In [8]:
# Identify the common headers
common_headers = set(df_tn01_99.columns).intersection(df_tn16_99.columns)

# Merge the DataFrames
df_tn01_tn16 = dd.merge(df_tn01_99, df_tn16_99, on=list(common_headers), how='outer', 
                        suffixes=('_dup_tn01', '_dup_tn16'))

# Compute the concatenated DataFrame to get the result
try:
    df_tn01_tn16_merge = df_tn01_tn16.compute()
    print("Merging DataFrame computed successfully.")
except Exception as e:
    print(f"Error in merging DataFrame: {e}")
    
print(df_tn01_tn16_merge.info())
df_tn01_tn16_merge.head()

Merging DataFrame computed successfully.
<class 'pandas.core.frame.DataFrame'>
Index: 5369309 entries, 0 to 64561
Columns: 470 entries, MaskID to Visit_dup12
dtypes: string(470)
memory usage: 11.7 GB
None


,MaskID,Visit_Dt,Visit,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,_15Aug2011ConsentDateDay,_15Aug2011ConsentDateMonth,_15Aug2011ConsentDateYear,Fall2017ConsentDateDay,...,Visit_Dt_dup9,Visit_dup9,TN16ProtocolDeviation,ProtocolDeviationOtherSubcateg,IfMiscellaneousSpecify,SubjectRandomizedInelgibilityA,Visit_Dt_dup10,Visit_dup10,SelectTheVersionOfTN16,Visit_dup12
0,968661,09/04/2012,PRN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,969347,10/08/2009,PRN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,971693,12/11/2019,PRN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,972762,02/28/2019,PRN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,972892,01/22/2007,PRN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [9]:
df_tn01_tn16_merge.to_feather('tn01_tn16_merge.feather')

In [7]:
import dask.dataframe as dd
import pandas as pd

# Replace 'your_file.feather' with the path to your Feather file
feather_file = 'tn01_tn16_merge.feather'

# Read the Feather file into a Pandas DataFrame
pandas_df = pd.read_feather(feather_file)

# Convert the Pandas DataFrame to a Dask DataFrame
df_tn01_tn16_merge = dd.from_pandas(pandas_df, npartitions=50) # You can adjust npartitions based on your dataset and system

# Now dask_df is a Dask DataFrame


In [8]:
df_tn01_tn16_merge.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 470 entries, MaskID to Visit_dup12
dtypes: string(470)

In [9]:
for column in df_tn01_tn16_merge.columns:
    df_tn01_tn16_merge[column] = df_tn01_tn16_merge[column].astype('category')


In [10]:
df_tn01_tn16_merge.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 470 entries, MaskID to Visit_dup12
dtypes: category(470)

In [14]:
for column in df_tn19_99.columns:
    df_tn19_99[column] = df_tn19_99[column].astype('category')
df_tn19_99.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 295 entries, MaskID to ExperiencedAnyHypoglycemicEven
dtypes: category(295)

In [18]:
import pandas as pd

# Calculate the percentage of missing values
missing_percentage = df_tn01_tn16_merge.isnull().mean() * 100

# Initialize a list to hold the data for the new DataFrame
data = []

# Iterate through each column in the original DataFrame
for column in df_tn01_tn16_merge.columns:
    # Find the top 5 most frequent values (excluding NaN) and their counts
    top_5_values_counts = df_tn01_tn16_merge[column].value_counts(dropna=True).head(5)
    top_5_values = top_5_values_counts.index.tolist()
    top_5_counts = top_5_values_counts.tolist()

    # Ensure the lists have 5 elements by padding with None if necessary
    top_5_values += [None] * (5 - len(top_5_values))
    top_5_counts += [None] * (5 - len(top_5_counts))

    # Determine the data type of the column
    column_dtype = str(df_tn01_tn16_merge[column].dtype)

    # Append the data for this column to the list
    data.append([missing_percentage[column], column_dtype] + top_5_values + top_5_counts)

# Define the columns for the new DataFrame
columns = ['Missing Percentage', 'Value Type', 
           'Top 1', 'Top 2', 'Top 3', 'Top 4', 'Top 5',
           'Top 1 Value Count', 'Top 2 Value Count', 'Top 3 Value Count', 
           'Top 4 Value Count', 'Top 5 Value Count']

# Create the new DataFrame
summary_df = pd.DataFrame(data, index=df_tn01_tn16_merge.columns, columns=columns)

# Sort the DataFrame by 'Missing Percentage' in descending order
summary_df = summary_df.sort_values(by='Missing Percentage', ascending=False)

# Save the DataFrame to a CSV file
summary_df.to_csv('TN_01_16_summary.csv')


In [ ]:
# Identify the common headers
common_headers_01_16_19= set(df_tn01_tn16_merge.columns).intersection(df_tn19_99.columns)

print('common_headers_01_16_19: ', len(common_headers_01_16_19))

# Merge the DataFrames
df_tn01_tn16_tn19 = dd.merge(df_tn01_tn16_merge, df_tn19_99, on=list(common_headers_01_16_19), how='outer', 
                        suffixes=('','_dup_tn19'))

# Compute the concatenated DataFrame to get the result
try:
    df_tn01_tn16_tn19_merge = df_tn01_tn16_tn19.compute()
    print("Merging DataFrame computed successfully.")
except Exception as e:
    print(f"Error in merging DataFrame: {e}")
    
print(df_tn01_tn16_tn19_merge.info())
df_tn01_tn16_tn19_merge.head()

common_headers_01_16_19:  99


In [ ]:
df_tn19_99.info()

In [15]:

file_path = 'TN_01_Merged_Data.csv'

# Open the file and read the first line
with open(file_path, 'r') as file:
    first_line = file.readline().strip()

# Split the first line into column headers
column_headers = first_line.split(',')

for i in tn_01_col_99:
    if i not in column_headers:
        print(i)

In [ ]:
import pandas as pd

# Names of the Excel files
file_names = ['TN_01_column_headers.csv', 'TN_16_column_headers.csv', 'TN_19_column_headers.csv', 'TN_20_column_headers.csv']
column_name = 'Column Header'  # Replace 'header' with the actual name of your column

# Reading the specified column from each file and storing it in a list
all_values = []
for file in file_names:
    df = pd.read_csv(file)
    if column_name in df.columns:
        all_values.extend(df[column_name].tolist())
    else:
        print(f"Column '{column_name}' not found in {file}")

# Removing duplicates
unique_values = list(set(all_values))

print(len(unique_values))
print(unique_values)

In [2]:
import pandas as pd

def prepare_and_save_to_feather(df, file_name):
    """
    Prepares a DataFrame for saving to a Feather file by ensuring consistent data types
    and handling NaN values, then saves it to the specified file.

    Parameters:
    df (pd.DataFrame): The DataFrame to be saved.
    file_name (str): The name of the Feather file to save.
    """
    # Ensure all columns are of type 'str' to avoid mixed types.
    # You can adjust this logic based on your specific data type needs.
    for col in df.columns:
        try:
            df[col] = df[col].astype(str)
        except Exception as e:
            print(f"Error converting column {col}: {e}")
            # Handle specific errors or fill NaN values if necessary
            df[col] = df[col].fillna('Unknown').astype(str)

    # Try saving the DataFrame to a Feather file
    try:
        df.to_feather(file_name)
        print(f"DataFrame successfully saved to {file_name}")
    except Exception as e:
        print(f"Error saving DataFrame to Feather: {e}")


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
tn_01_header

# 1. Load merged datasets

## 1.1 TN_01: Merging into one

### 1.1.1 Data type transformation: string into category, float64 into float16

In [3]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# The following code is to merge the two datasets for TN_01, but will cause error or dead kernel. 
# Convert the 'ProtocolVersion' column to string in both DataFrames
file_path_1 = "TN_01_merged_data_1.csv"
file_path_2 = "TN_01_merged_data_23.csv"

merged_first_part = pd.read_csv(file_path_1)
merged_data_23 = pd.read_csv(file_path_2)

In [4]:
# Example usage:
prepare_and_save_to_feather(merged_data_23, "TN_01_merged_data_23.feather")

DataFrame successfully saved to TN_01_merged_data_23.feather


In [ ]:
# Example usage:
prepare_and_save_to_feather(merged_first_part, "TN_01_merged_data_1.feather")

In [2]:
merged_data_23.info()

NameError: name 'merged_data_23' is not defined

In [3]:
print(merged_first_part.info())
print()
print(merged_data_23.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519750 entries, 0 to 519749
Columns: 1563 entries, MaskID to SignedConsentFor2017Protocol
dtypes: float64(665), int64(1), object(897)
memory usage: 6.1+ GB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968807 entries, 0 to 1968806
Data columns (total 17 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   MaskID                         int64  
 1   Date_of_Registration           object 
 2   Status                         object 
 3   Date_of_Study_Start            object 
 4   Date_of_Draw                   object 
 5   Event_Title                    object 
 6   Spec_Name                      object 
 7   SampleMaskID                   object 
 8   Test_Name                      object 
 9   Result                         object 
 10  Result_Type                    object 
 11  Visit                          object 
 12  Date_at_Test_Results_Reported  object 
 13  Date_at_Ev

In [4]:
print(merged_first_part.memory_usage(deep=True).sum())
print()
print(merged_data_23.memory_usage(deep=True).sum())

17883089368

1802078130


##### By changing some columns from float64 to float 16. I checked the accuracy loss, which is minimal and negligible. 

In [ ]:
import pandas as pd
import numpy as np

def transform_data_types(df):
    for column in df.columns:
        # Check for NaN values in the column
        has_nan = df[column].isnull().any()

        if df[column].dtype == 'float64':
            # Convert to int16 if all values are integers and there are no NaN values
            if not has_nan and all(df[column].dropna().apply(float.is_integer)):
                df[column] = df[column].astype('int16')
            else:
                # Convert to float16 otherwise
                df[column] = df[column].astype('float16')

        elif df[column].dtype == 'object':
            # Convert string columns to category
            if all(df[column].apply(lambda x: isinstance(x, str) or pd.isnull(x))):
                df[column] = df[column].astype('category')

    return df

In [6]:
merged_first_part = transform_data_types(merged_first_part)
merged_second_part = transform_data_types(merged_data_23)

print(merged_first_part.memory_usage(deep=True).sum())
print()
print(merged_second_part.memory_usage(deep=True).sum())

1311209169

245206343


In [7]:
print(merged_first_part.info())
print()
print(merged_second_part.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519750 entries, 0 to 519749
Columns: 1563 entries, MaskID to SignedConsentFor2017Protocol
dtypes: category(888), float16(665), int64(1), object(9)
memory usage: 1.1+ GB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968807 entries, 0 to 1968806
Data columns (total 17 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   MaskID                         int64   
 1   Date_of_Registration           category
 2   Status                         category
 3   Date_of_Study_Start            category
 4   Date_of_Draw                   category
 5   Event_Title                    category
 6   Spec_Name                      category
 7   SampleMaskID                   category
 8   Test_Name                      category
 9   Result                         object  
 10  Result_Type                    category
 11  Visit                          category
 12  Date_at_Test_Results_Report

### 1.2.2 Data merge into one

In [8]:
common_headers_123 = set.intersection(set(merged_first_part.columns), set(merged_second_part.columns))

# Merge the first two DataFrames on common headers using outer join
TN_01_merged_data = pd.merge(merged_first_part, merged_second_part, on=list(common_headers_123), how='outer', 
                            suffixes=('_dup_x', '_dup_y'))

print(TN_01_merged_data.info())
TN_01_merged_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2923848 entries, 0 to 2923847
Columns: 1578 entries, MaskID to LabID
dtypes: category(900), float16(666), int64(1), object(11)
memory usage: 6.4+ GB
None


,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,_22Jul2009ConsentSignedDay,_22Jul2009ConsentSignedMonth,_22Jul2009ConsentSignedYear,...,Spec_Name,SampleMaskID,Test_Name,Result,Result_Type,Date_at_Test_Results_Reported,Date_at_Evaluation,Date_Received,Date_Shipped,LabID
0,200023,07/01/2015,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,Serum - Autoantibodies,S3827078,GAD65H,0,RPTD,07/20/2015,07/14/2015,07/14/2015,07/08/2015,547.0
1,200023,07/01/2015,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,Serum - Autoantibodies,S3827078,MIAA,0,RPTD,07/20/2015,07/14/2015,07/14/2015,07/08/2015,547.0
2,200023,07/01/2015,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,Serum - Autoantibodies,S3827078,IA-2H,0,RPTD,07/20/2015,07/14/2015,07/14/2015,07/08/2015,547.0
3,200071,01/24/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200071,01/14/2019,PRN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
TN_01_merged_data.head()

NameError: name 'TN_01_merged_data' is not defined

In [ ]:
import pandas as pd

def prepare_and_save_to_feather(df, file_name):
    """
    Prepares a DataFrame for saving to a Feather file by ensuring consistent data types
    and handling NaN values, then saves it to the specified file.

    Parameters:
    df (pd.DataFrame): The DataFrame to be saved.
    file_name (str): The name of the Feather file to save.
    """
    # Ensure all columns are of type 'str' to avoid mixed types.
    # You can adjust this logic based on your specific data type needs.
    for col in df.columns:
        try:
            df[col] = df[col].astype(str)
        except Exception as e:
            print(f"Error converting column {col}: {e}")
            # Handle specific errors or fill NaN values if necessary
            df[col] = df[col].fillna('Unknown').astype(str)

    # Try saving the DataFrame to a Feather file
    try:
        df.to_feather(file_name)
        print(f"DataFrame successfully saved to {file_name}")
    except Exception as e:
        print(f"Error saving DataFrame to Feather: {e}")

# Example usage:
prepare_and_save_to_feather(TN_01_merged_data, "TN_01_merged_data.feather")


In [13]:
TN_01_merged_data['OnlineScreeningReferralSource'] = TN_01_merged_data['OnlineScreeningReferralSource'].astype(str)
TN_01_merged_data['DPT1ParticipantIDNumber'] = TN_01_merged_data['DPT1ParticipantIDNumber'].astype(str)
TN_01_merged_data['SelectStudyForPreventionOfT1D'] = TN_01_merged_data['SelectStudyForPreventionOfT1D'].astype(str)

TN_01_merged_data.to_feather("TN_01_merged_data.feather")

ArrowTypeError: ("Expected bytes, got a 'float' object", 'Conversion failed for column TestingPlaceForAutoAntibodyOut with type object')

#### The merged dataset is too large to be saved in the working directory. We are to turn columns with numerical values and large portion of missing values into sparse data structure.

In [2]:
summary_df_tn_01 = pd.read_csv('merged_data_summary.csv')
summary_df_tn_01.info()

NameError: name 'pd' is not defined

### 1.2.4 Numerical data structure transformation

In [ ]:
# Filter based on Missing Percentage and Value Type
filtered_columns = summary_df_tn_01[(summary_df_tn_01['Missing Percentage'] > 85) & 
                             (summary_df_tn_01['Value Type'].isin(['float16', 'int64']))]

# Get the list of headers from the filtered DataFrame
headers_with_missing_85 = filtered_columns['header'].tolist()

# Print the count of columns
print("Count of columns with missing Percentage > 85% and Value Type 'float16' or 'int64':", len(headers_with_missing_85))

#### Tried many times and found that: 1) for columns with very high missing values, sparse data structure significantly save memory; 2) when missing value less than 85%, sparse data structure does have advantage over float16. 

In [ ]:
print(TN_01_merged_data.info())
print()
print(TN_01_merged_data.memory_usage(deep=True).sum())

In [ ]:
error_columns = []  # List to store columns that raised errors

for col in headers_with_missing_85:
    try:
        print("Before transformation of", col, ":", TN_01_merged_data[col].memory_usage(deep=True), "bytes")
        
        # Check data type of the column
        col_dtype = TN_01_merged_data[col].dtype
        print(col_dtype)
        if col_dtype == 'float16' or col_dtype == 'int64':
            # Convert numeric columns to default sparse type
            TN_01_merged_data[col] = pd.to_numeric(TN_01_merged_data[col], errors='coerce').astype('Sparse[float16]')
        # No need to replace the column in the original DataFrame
        # Just printing out the memory usage after transformation
        print("After transformation to Sparse of", col, ":", TN_01_merged_data[col].memory_usage(deep=True), "bytes")
    except ValueError as e:
        print(f"Error converting column {col}: {e}")
        error_columns.append(col)

    print()

# Print out columns that had errors
if error_columns:
    print("Columns with errors:", error_columns)

In [ ]:
print(TN_01_merged_data.info())
print()
print(TN_01_merged_data.memory_usage(deep=True).sum())

In [ ]:
for column in TN_01_merged_data.columns:
    if TN_01_merged_data[column].dtype == 'object':
        TN_01_merged_data[column] = TN_01_merged_data[column].astype('category')

In [ ]:
#TN_01_merged_data = TN_01_merged_data.drop('Unnamed: 0', axis=1)

In [ ]:
print(TN_01_merged_data.info())
print()
print(TN_01_merged_data.memory_usage(deep=True).sum())

#### change MaskID from int64 to int32

In [ ]:
test_col = TN_01_merged_data['MaskID']
print(test_col.memory_usage(deep=True))
print(test_col.head(30))
print()
test_col_16 = test_col.astype('int16')
print(test_col_16.memory_usage(deep=True))
print(test_col_16.head(30))
print()
test_col_32 = test_col.astype('Int32')
print(test_col_32.memory_usage(deep=True))
print(test_col_32.head(30))
print()

In [ ]:
TN_01_merged_data['MaskID'] = TN_01_merged_data['MaskID'].astype('Int32')

print(TN_01_merged_data.info())
print()
print(TN_01_merged_data.memory_usage(deep=True).sum())

In [ ]:
unique_values = TN_01_merged_data['D5MeterReadingResult4_1'].unique()
print(unique_values)

#### turn columns as Sparse[float64] to Sparse[float16]

In [ ]:
# Iterate over each column and convert if it's of type Sparse[float64]
for column in TN_01_merged_data.columns:
    if pd.api.types.is_sparse(TN_01_merged_data[column].dtype):
        if TN_01_merged_data[column].dtype.subtype == np.float64:
            TN_01_merged_data[column] = TN_01_merged_data[column].astype('Sparse[float16]')

print(TN_01_merged_data.info())
print()
print(TN_01_merged_data.memory_usage(deep=True).sum())

#### turn columns as float16 to category

In [ ]:
test_col = TN_01_merged_data['Race_White']

print("memory used as float16: ",test_col.memory_usage(deep=True))
print()

# Convert to float32 or float64 before converting to category
test_col_float32 = test_col.astype('float32')
print("memory used as float32: ",test_col_float32.memory_usage(deep=True))
print()

test_col_category = test_col_float32.astype('category')
print("memory used as category: ", test_col_category.memory_usage(deep=True))


In [ ]:
# Iterate over each column and convert if it's of type float16
for column in TN_01_merged_data.columns:
    if TN_01_merged_data[column].dtype == np.float16:
        print("memory used as float16: ", TN_01_merged_data[column].memory_usage(deep=True))
        #convert to float32 first
        TN_01_merged_data[column] = TN_01_merged_data[column].astype('float32')
        print("memory used as float32: ", TN_01_merged_data[column].memory_usage(deep=True))
        # Convert the column to 'category'
        TN_01_merged_data[column] = TN_01_merged_data[column].astype('category')
        print("memory used as category: ", TN_01_merged_data[column].memory_usage(deep=True))
        print()

# Display DataFrame info and memory usage
print(TN_01_merged_data.info())
print()
print(TN_01_merged_data.memory_usage(deep=True).sum())


#### Turn SampleMaskID into integer

In [ ]:
# SampleMaskID has the biggest memory
# To turn it into interger, we need to check whether all values are starting with 'S', and the rest of values are integer.
column = TN_01_merged_data['SampleMaskID']

print("memory as category: ", column.memory_usage(deep=True))

test_col_obj = test_col.astype('object')
print("memory as object: ",test_col_obj.memory_usage(deep=True))

# Filter out missing values
non_missing_column = column[column.notna()]

# Check if values start with 'S'
starts_with_s = non_missing_column.str.startswith('S')

# Check if the rest part of the values are integers
rest_is_integer = non_missing_column.str.lstrip('S').apply(lambda x: x.isdigit())

# Combine the conditions
all_conditions_met = starts_with_s & rest_is_integer

# Check if all non-missing values in the column meet the conditions
all_values_meet_conditions = all_conditions_met.all()

print("Do all non-missing values start with 'S' and the rest part are integers?", all_values_meet_conditions)

##### No.1 change in column values: remove 'S' for 'SampleMaskID'

In [ ]:
# Strip 's' from the start of each string in the column
TN_01_merged_data['SampleMaskID'] = TN_01_merged_data['SampleMaskID'].str.lstrip('S')
print("original memory usage: ", TN_01_merged_data['SampleMaskID'].memory_usage(deep=True))

In [ ]:
TN_01_merged_data['SampleMaskID'].head(30)

In [ ]:
# SexOfRelativeWithT1D14_1 as category in the dataframe with only one value: Male.  
test_col = TN_01_merged_data['SampleMaskID'] 

print("original memory usage: ", test_col.memory_usage(deep=True))
test_col_32 = test_col.astype('Int32')
print("memory usage as Int32: ", test_col_32.memory_usage(deep=True))
print(test_col_32.head(30))

test_col_16 = test_col.astype('Int16')
print("memory usage as Int16: ", test_col_16.memory_usage(deep=True))
print(test_col_16.head(30))

In [ ]:
# Convert the column to nullable integer type Int32
TN_01_merged_data['SampleMaskID'] = TN_01_merged_data['SampleMaskID'].astype('Int32')
print("memory as Int32: ", TN_01_merged_data['SampleMaskID'].memory_usage(deep=True))

In [ ]:
# Display DataFrame info and memory usage
print(TN_01_merged_data.info())
print()
print(TN_01_merged_data.memory_usage(deep=True).sum())

In [ ]:
top_5_values = TN_01_merged_data['SampleMaskID'].value_counts().nlargest(5)
print(top_5_values)

#### turn some category into Sparse[str]

In [ ]:
# SexOfRelativeWithT1D14_1 as category in the dataframe with only one value: Male.  
test_col = TN_01_merged_data['SexOfRelativeWithT1D14_1']

print(test_col.memory_usage(deep=True))
print()

test_col_category = test_col.astype('Sparse[str]')
print(test_col_category.memory_usage(deep=True))

#### It is an unefficient way to turn string values into sparse even when the column with nearly 100% missing values. 

#### Function to check memory usage of a dataframe

In [ ]:
import pandas as pd

# Assuming merged_data_all is your DataFrame

def column_memory_usage(df):
    # List to store memory usage and data types
    memory_info = []

    # Iterating through each column to get memory usage and data type
    for col in df.columns:
        memory = df[col].memory_usage(deep=True)
        dtype = df[col].dtype
        memory_info.append({'Column': col, 'Memory Usage (Bytes)': memory, 'Data Type': dtype})

    # Create a DataFrame from the list
    memory_df = pd.DataFrame(memory_info)

    # Sorting by memory usage in descending order
    memory_df = memory_df.sort_values(by='Memory Usage (Bytes)', ascending=False)

    return memory_df

In [ ]:
# Call the function and store the result
column_memory_info = column_memory_usage(TN_01_merged_data)

# Display the result
print(column_memory_info.info())

column_memory_info.to_csv('TN_01_memory_usage_2.csv', index = False)

## 1.2 TN_16

In [ ]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:
# The following code is to load 
# load the merged data of TN_16
file_path_16 = "TN_16_merged_data.csv"

TN_16_merged_data = pd.read_csv(file_path_16)

print(TN_16_merged_data.info())
print(TN_16_merged_data.memory_usage(deep=True).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445461 entries, 0 to 2445460
Columns: 262 entries, MaskID to LabID
dtypes: float64(99), object(163)
memory usage: 4.8+ GB
None
19758580453


In [ ]:
import pandas as pd

def prepare_and_save_to_feather(df, file_name):
    """
    Prepares a DataFrame for saving to a Feather file by ensuring consistent data types
    and handling NaN values, then saves it to the specified file.

    Parameters:
    df (pd.DataFrame): The DataFrame to be saved.
    file_name (str): The name of the Feather file to save.
    """
    # Ensure all columns are of type 'str' to avoid mixed types.
    # You can adjust this logic based on your specific data type needs.
    for col in df.columns:
        try:
            df[col] = df[col].astype(str)
        except Exception as e:
            print(f"Error converting column {col}: {e}")
            # Handle specific errors or fill NaN values if necessary
            df[col] = df[col].fillna('Unknown').astype(str)

    # Try saving the DataFrame to a Feather file
    try:
        df.to_feather(file_name)
        print(f"DataFrame successfully saved to {file_name}")
    except Exception as e:
        print(f"Error saving DataFrame to Feather: {e}")

# Example usage:
prepare_and_save_to_feather(TN_16_merged_data, "TN_16_merged_data.feather")


In [ ]:
dtypes={'AECausalityByReviewer': 'object',
       'AEOccurDtDay': 'float64',
       'AEOccurDtMonth': 'float64',
       'AEOccurDtYear': 'float64',
       'AEReasonForFollowup': 'object',
       'AEReportDtDay': 'float64',
       'AEReportDtMonth': 'float64',
       'AEReportDtYear': 'float64',
       'AEReporterStaffCode': 'float64',
       'AEResolveDtDay': 'float64',
       'AEResolveDtMonth': 'float64',
       'AEResolveDtYear': 'float64',
       'AEReviewPriorSimilarSAEs': 'object',
       'AEReviewReportingTimeframe': 'object',
       'AEReviewTreatmentUnblindedForC': 'object',
       'AdverseEventID': 'float64',
       'AreThere3GlucoseValuesAvailabl': 'object',
       'AssessDateDay': 'float64',
       'AssessDateYear': 'float64',
       'AvgAllRecordedGlucosesMmoll': 'object',
       'AvgRecordedFastingGlucosesmmol': 'object',
       'ConMedsFrequencyOtherDetail': 'object',
       'ConMedsIntervalOtherDetails': 'object',
       'DateRecordedMM1': 'object',
       'DateRecordedMM2': 'object',
       'DateRecordedMM3': 'object',
       'DateStatusChangeDay': 'float64',
       'DateStatusChangeYear': 'float64',
       'DoWDay': 'float64',
       'DoWYear': 'float64',
       'Dose': 'float64',
       'HighestGlucosemmol': 'object',
       'IfMiscellaneousSpecify': 'object',
       'InsulDoseInfoAvailableFor3Days': 'object',
       'InsulinDoseLast24Hrs': 'float64',
       'InsulinRoutineDaily': 'object',
       'LastHbA1cDay': 'float64',
       'LastHbA1cYear': 'float64',
       'LowestGlucosemmoll': 'object',
       'MMTTCollected': 'object',
       'OGTTCollected': 'object',
       'PInotified': 'object',
       'ProtcolDeviationMonth': 'object',
       'ProtocolDeviationOtherSubcateg': 'object',
       'ProtocolVersion': 'object',
       'RequiredSpecimensCollected': 'object',
       'Result': 'object',
       'StartDateDay': 'float64',
       'StartDateYear': 'float64',
       'StopDateMonth': 'object',
       'SubjectRandomizedInelgibilityA': 'object',
       'SubjectUsingCGMS': 'object',
       'TN16ProtocolDeviation': 'object',
       'Visit_Dt_dup1': 'object',
       'Visit_Dt_dup9': 'object',
       'Visit_dup1': 'object',
       'Visit_dup9': 'object',
       'WeightKg': 'float64',
       'severity': 'float64',
       'AECategory': 'object',
       'LabID': 'float64',
       'Eventabateafterstoppingdrug': 'object',
       'Date_of_Study_Start': 'object',
        'AECausalityByReporter' : 'object',
        'Current_Site_Number' : 'object',
        'ReasonForWithdrawal': 'object',
        'Eventreappearafterreintroducti' : 'object',
        'AETerm' : 'object',
        'AEExpected1_1' : 'object',
        'AEPrimarySecondary1_1' : 'object',
        'AEReportType' : 'object',
        'AEPatientOutcome1_1': 'object',
        'WillingToBeContacted': 'object',
        'Old_Site_Number': 'object',
        'Date_of_Registration': 'object',
        'ChangeOfStatusOccuring': 'object',
        'AETreatLocation': 'object',
        'DateStatusChangeMonth': 'object',
        'Reason_for_Transfer': 'object',
        'Status': 'object',
        'DoWMonth': 'object'    
       }

In [ ]:
dtype={'AEAdditionalInfoRequested': 'object',
       'AESerious': 'object',
       'AETreatLocationOther': 'object',
       'AddisonsDiseaseWithinLast6Months': 'object',
       'AddisonsDiseaseWithinLast6Months_dup16': 'object',
       'AdmissionMonth': 'object',
       'AllergiesWithinLast6Months': 'object',
       'AllergiesWithinLast6Months_dup16': 'object',
       'AlopeciaWithinLast6Months': 'object',
       'AlopeciaWithinLast6Months_dup16': 'object',
       'AlphaGlucosidaseInhibitors': 'object',
       'AlphaGlucosidaseInhibitors_dup16': 'object',
       'AnionGapMonth': 'object',
       'AnionGapUnits': 'object',
       'AntibodySampleCollected_dup15': 'object',
       'AntibodySampleCollected_dup16': 'object',
       'AsthmaWithinLast6Months': 'object',
       'AsthmaWithinLast6Months_dup16': 'object',
       'BasophilsResultsNormal': 'object',
       'BetaBlockers_dup15': 'object',
       'BetaBlockers_dup16': 'object',
       'BiweeklyInjectionVisitSchedule': 'object',
       'BllodForDNASampleCollected_dup15': 'object',
       'BllodForDNASampleCollected_dup16': 'object',
       'BloodForCellsSampleCollected_dup15': 'object',
       'BloodForCellsSampleCollected_dup16': 'object',
       'BloodForRNASampleCollected_dup15': 'object',
       'BloodForRNASampleCollected_dup16': 'object',
       'CancerWithinLast6Months': 'object',
       'CancerWithinLast6Months_dup16': 'object',
       'CeliacDiseaseWithinLast6Months': 'object',
       'CeliacDiseaseWithinLast6Months_dup16': 'object',
       'CentralIRBOrlocalIRB': 'object',
       'CentralIRBOrlocalIRB_dup26': 'object',
       'ColitisOrColonProblemsWithinLa_dup15': 'object',
       'ColitisOrColonProblemsWithinLa_dup16': 'object',
       'ConcernsAboutReceivingPlaceboR': 'object',
       'ConflictingResponsibilitiesRan': 'object',
       'CongenitalHeartDiseaseWithinLa_dup15': 'object',
       'CongenitalHeartDiseaseWithinLa_dup16': 'object',
       'ConsenToContinuedStorageOfBloo_dup15': 'object',
       'ConsenToContinuedStorageOfBloo_dup16': 'object',
       'Corticosteroids_dup15': 'object',
       'Corticosteroids_dup16': 'object',
       'CriteriaParticipantDiagnosewit': 'object',
       'CurrentPhase': 'object',
       'D1FastingPlasmaGlucMM1_1': 'object',
       'D1FastingPlasmaGlucUnits1_1': 'object',
       'D2b2HourGlucMM1_1': 'object',
       'D2b2HourGlucUnits1_1': 'object',
       'D3RandomPlasmaGlucMM1_1': 'object',
       'D3RandomPlasmaGlucUnits1_1': 'object',
       'D4HbA1cMM1_1': 'object',
       'D5MeterReadingLoc1_1': 'object',
       'D5MeterReadingMM1_1': 'object',
       'D5MeterReadingUnit1_1': 'object',
       'D5OtherTestResultMM1_1': 'object',
       'D5OtherTestResultMM2_1': 'object',
       'D6OtherTestResultUnit2_1': 'object',
       'DateCompletionMM': 'object',
       'DateConsentSignedMonth_dup15': 'object',
       'DateControlBecameAutoantibodyP2': 'object',
       'DateOfBloodDrawMonth': 'object',
       'DateParticipantwascontacted1_2': 'object',
       'DateReportedMonth': 'object',
       'DateSignedNEWInformedConsentFo2_dup15': 'object',
       'DateSignedNEWInformedConsentFo2_dup16': 'object',
       'DateofAssessmentMM': 'object',
       'DatestudyendpointmetMM': 'object',
       'DiagnosisBy': 'object',
       'DiagnosisByOthersSpecify': 'object',
       'DiagnosisMonth': 'object',
       'Didtheparticipantindicatetheym': 'object',
       'Diphenylhydantoin_dup15': 'object',
       'Diphenylhydantoin_dup16': 'object',
       'DischargeMonth': 'object',
       'DoRMonth': 'object',
       'DoesNotTolerateOGTTRank': 'object',
       'DoesTheParticipantWishToContin': 'object',
       'E1aDKA': 'object',
       'E1bDateOfOnsetMM': 'object',
       'E1bPolyuria': 'object',
       'E1cDateofOnsetMM': 'object',
       'E1cPolydipsia': 'object',
       'E1dPolyphagia': 'object',
       'E1eUnexplainedWeightLoss': 'object',
       'EosinophilsResultsNormal': 'object',
       'EpilepsyConvulsionsSeizuresWit_dup15': 'object',
       'EpilepsyConvulsionsSeizuresWit_dup16': 'object',
       'F1DateStudyEndpointMetMM': 'object',
       'F2ClinicalDiagnosisDateMM': 'object',
       'FamilyWishesToWaitUntilOlderRa': 'object',
       'Fatigue': 'object',
       'FatigueMonth': 'object',
       'FearOfStudyDrugRisksRank': 'object',
       'FinalContactAttempt1_1': 'object',
       'Followupstatuscode': 'object',
       'GlucoseMonth1_1': 'object',
       'GlucoseMonth2_1': 'object',
       'GlucoseMonth3_1': 'object',
       'GlucoseMonth4_1': 'object',
       'GlucoseType1_1': 'object',
       'GlucoseType2_1': 'object',
       'GlucoseType3_1': 'object',
       'GlucoseType4_1': 'object',
       'H1OtherSymptoms': 'object',
       'H2OvernightHospitalization': 'object',
       'H3AdmissionToICU': 'object',
       'H4DKA': 'object',
       'H5StartedInsulin': 'object',
       'Hastheparticipantbeentestedfor': 'object',
       'Hastheparticipantbeentoahealth': 'object',
       'Hastheparticipantexperiencedvo': 'object',
       'Hastheparticipanthadacough': 'object',
       'Hastheparticipanthadafever': 'object',
       'Hastheparticipanthadalossreduc': 'object',
       'Hastheparticipanthadanydifficu': 'object',
       'Hastheparticipanthadanypersist': 'object',
       'Hastheparticipanthadchillsasso': 'object',
       'Hastheparticipanthadknownconta': 'object',
       'Hastheparticipanthadknownconta2': 'object',
       'Hasthesubjectexperiencedblurre': 'object',
       'Hasthesubjectexperiencedpolydy': 'object',
       'Hasthesubjectexperiencedpolyur': 'object',
       'Hasthesubjectexperienceduninte': 'object',
       'Hasthesubjectprovidedcontinued': 'object',
       'Havetherebeenanychangesinthepa': 'object',
       'HbA1cMonth': 'object',
       'HbA1cSamplesCollected_dup15': 'object',
       'HbA1cSamplesCollected_dup16': 'object',
       'Height': 'object',
       'Height_dup16': 'object',
       'HematocritResultsNormal': 'object',
       'HemoglobinResultAbormalClinica': 'object',
       'HemoglobinResultsNormal': 'object',
       'HepatitisOrLiverDiseaseWithinL_dup15': 'object',
       'HepatitisOrLiverDiseaseWithinL_dup16': 'object',
       'HighBPwithinLast6Months': 'object',
       'HighBPwithinLast6Months_dup16': 'object',
       'HighCholesterolWithinLast6Months': 'object',
       'HighCholestrolWithinLast6Months': 'object',
       'HospitalizedAtDiagnosisTime': 'object',
       'IfYesOutcomeofCOVID19Test': 'object',
       'IfyesDateCOVID19TestAdminister2': 'object',
       'ImmunosuppressiveTherapies_dup15': 'object',
       'ImmunosuppressiveTherapies_dup16': 'object',
       'IndicateTheStudyParticipantWas': 'object',
       'IndicateTheStudyParticipantWas_dup9': 'object',
       'InfectiousMononucleosisWithinL_dup15': 'object',
       'InfectiousMononucleosisWithinL_dup16': 'object',
       'InflammatoryBowelDiseaseWithin_dup15': 'object',
       'InflammatoryBowelDiseaseWithin_dup16': 'object',
       'Informationgatheredfrom': 'object',
       'InsulinTreatmentStartedMonth': 'object',
       'IsParticipantCurrentlyEligible': 'object',
       'IsParticipantPregnant': 'object',
       'IsParticipantPregnant_dup16': 'object',
       'IsTheParticipantInterested': 'object',
       'Istheparticipantinterestedinth': 'object',
       'Ketoacidosis': 'object',
       'LimitedLifeExpectancyOrImmunos': 'object',
       'LimitedLifeExpectancyOrImmunos_dup16': 'object',
       'LymphocytesResultsNormal': 'object',
       'MCHCResultAbormalClinicallySig': 'object',
       'MCHCresultsNormal': 'object',
       'MCHResultAbormalClinicallySign': 'object',
       'MCHresultsNormal': 'object',
       'MCVResultAbormalClinicallySign': 'object',
       'MCVresultsNormal': 'object',
       'MeasuredBy1_1': 'object',
       'MeasuredBy2_1': 'object',
       'MeasuredBy3_1': 'object',
       'MeasuredBy4_1': 'object',
       'MechanisticSampleCollectionMon_dup15': 'object',
       'MechanisticSampleCollectionMon_dup16': 'object',
       'Meglinitides': 'object',
       'Meglitinides': 'object',
       'Metformin': 'object',
       'Metformin_dup16': 'object',
       'MethodofContact': 'object',
       'MonocytesResultAbormalClinical': 'object',
       'MonocytesResultsNormal': 'object',
       'MonthlyInfusionVisitScheduleRa': 'object',
       'MultipleSclerosisWithinLast6Mo': 'object',
       'MultipleSclerosisWithinLast6Mo_dup16': 'object',
       'NeutrophilsResultsNormal': 'object',
       'Niacin_dup15': 'object',
       'Niacin_dup16': 'object',
       'NotificationStatus': 'object',
       'OGTTSampleCollected_dup15': 'object',
       'OGTTSampleCollected_dup16': 'object',
       'OtherAutoimmuneDiseasesWithinL_dup15': 'object',
       'OtherAutoimmuneDiseasesWithinL_dup16': 'object',
       'OtherGlucoseLoweringAgents': 'object',
       'OtherGlucoseLoweringAgents_dup16': 'object',
       'OtherRank': 'object',
       'ParticipantAgreeToContinue': 'object',
       'ParticipantContacted': 'object',
       'ParticipantDiagnosedWithT1D_dup15': 'object',
       'ParticipantDiagnosedWithT1D_dup16': 'object',
       'ParticipantExpectedToContinue': 'object',
       'ParticipantPregnant_dup15': 'object',
       'ParticipantPregnant_dup16': 'object',
       'ParticipantUsedInsulinOrOtherG_dup15': 'object',
       'ParticipantUsedInsulinOrOtherG_dup16': 'object',
       'ParticipantWasContactedByOther': 'object',
       'ParticipantWillingToBeContacte': 'object',
       'Participantisahotleadwarmleado': 'object',
       'PermissionForSampleStorage_dup15': 'object',
       'PerniciousAnemiaWithinLast6Mo': 'object',
       'PerniciousAnemiaWithinLast6Mo_dup16': 'object',
       'PlasmaGlucoseMonth': 'object',
       'PlasmaGlucoseUnits': 'object',
       'PlasmaSampleCollected_dup15': 'object',
       'PlasmaSampleCollected_dup16': 'object',
       'PlateletCountResultsNormal2': 'object',
       'Pleaseselecttypeofcontactattem': 'object',
       'Polydipsia': 'object',
       'PolydipsiaMonth': 'object',
       'Polyphagia': 'object',
       'PolyphagiaMonth': 'object',
       'Polyuria': 'object',
       'PolyuriaMonth': 'object',
       'PotassiumDepletingDiurectics_dup15': 'object',
       'PotassiumDepletingDiurectics_dup16': 'object',
       'PregnancyCompletionMonth': 'object',
       'ProtocolDeviation': 'object',
       'PsoriasisWithinLast6Months': 'object',
       'PsoriasisWithinLast6Months_dup16': 'object',
       'RBCResultAbormalClinicallySign': 'object',
       'RBCResultsNormal': 'object',
       'RecommendConsentChange': 'object',
       'RecommendProtocolChange': 'object',
       'RheumatologicWithinLast6Months': 'object',
       'RheumatologicWithinLast6Months_dup16': 'object',
       'SamplesCollectionMonth_dup15': 'object',
       'SamplesCollectionMonth_dup16': 'object',
       'SerumForProteomicsSampleCollec_dup15': 'object',
       'SerumForProteomicsSampleCollec_dup16': 'object',
       'SerumMechanisticSerumSampleCol': 'object',
       'SignedMonitoringConsentProtoco': 'object',
       'SignedMonitoringConsentProtoco_dup16': 'object',
       'SiteInvestigatorAckStudyEndPoi': 'object',
       'Spring2019ConsentDateMonth': 'object',
       'TN01ParticipantStatusSectionB': 'object',
       'TN16TN19TN20EligibilityStatus': 'object',
       'TN18Exclusion12bDoesthiscondit': 'object',
       'Thiazolidinediones': 'object',
       'Thiazolidinediones_dup16': 'object',
       'ThyroidDiseaseWithinLast6Months': 'object',
       'ThyroidDiseaseWithinLast6Months_dup16': 'object',
       'TimeCommitmentRank': 'object',
       'TypeofTestAdministered': 'object',
       'UlcerWithinLast6Months': 'object',
       'UlcerWithinLast6Months_dup16': 'object',
       'UnableUnwillingToTravelRank': 'object',
       'UnexplainedWeightLoss': 'object',
       'Units1_1': 'object',
       'Units2_1': 'object',
       'Units3_1': 'object',
       'Units4_1': 'object',
       'UnwillingToTakeInvestigational': 'object',
       'UrineKetonesMonth': 'object',
       'Visit_Type_dup15': 'object',
       'VistMissed': 'object',
       'VitiligoWithinLast6Months': 'object',
       'VitiligoWithinLast6Months_dup16': 'object',
       'WBCresultsNormal': 'object',
       'WasTheParticipantNotified': 'object',
       'WastheCOVID19diagnosissuspecte': 'object',
       'Wastheparticipanthospitalizedd': 'object',
       'WastheparticipantplacedintheIC': 'object',
       'Wastheparticipantplacedonavent': 'object',
       'Wastheparticipanttreatedwithme': 'object',
       'Weight': 'object',
       'Weight_dup16': 'object',
       'Whatwastheseverity': 'object',
       'Whendidthecoughoccur': 'object',
       'WhendidthesesymptomsOccur': 'object',
       'WhichProtocolVersion2': 'object',
       'WholeBloodPBMCPlasmaSampleColl': 'object',
       'WillingToContinueAfterPregnanc': 'object',
       '_22Jul2009ConsentSignedMonth': 'object',
       '_22Jul2009ScreeningInformedCon': 'object',
       '_22Jul2009VisitType': 'object',
       '_2WeekIfusionPeriodRank': 'object',
       '_2aIfyes': 'object',
       '_2dMailedLetter': 'object',
       '_3aIfyes': 'object',
       '_8Hastheparticipantbeendiagnos': 'object',
       'centralIRBscriteria': 'object',
       'centralIRBscriteria_dup26': 'object',
       'f1availableWeightMM': 'object',
       'f1availableWeightUnits': 'object',
       'f2availableWeightMM': 'object',
       'f2availableWeightUnits': 'object',
       'hyperglycemia': 'object',
       'hyperglycemia_dup16': 'object',
       'immunomodulatory': 'object',
       'immunomodulatory_dup16': 'object',
       'participantdiagnosedT1D': 'object',
       'participantdiagnosedT1D_dup16': 'object',
       'systemicsteroids': 'object',
       'systemicsteroids_dup16': 'object',
       'testName1_1': 'object',
       'testName2_1': 'object',
       'willingToConsiderAtALaterTime': 'object'}

In [2]:
import dask.dataframe as dd

ddf_16 = dd.read_csv("TN_16_merged_data.csv", dtype=str, assume_missing=True)

# pick the first 1 million rows
#ddf_16_1m = ddf_16.head(n=1000000, npartitions=-1)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [5]:

ddf_16.to_

<class 'dask.dataframe.core.DataFrame'>
Columns: 262 entries, MaskID to LabID
dtypes: string(262)

In [6]:
ddf_01 = dd.read_csv("TN_01_Merged_Data.csv", dtype=str, assume_missing=True)
# pick the first 1 million rows
#ddf_01_1m = ddf_01.head(n=1000000, npartitions=-1)

In [1]:
ddf_01.info()

NameError: name 'ddf_01' is not defined

In [1]:
merged_result.head()

NameError: name 'merged_result' is not defined

In [ ]:
import dask.dataframe as dd

# Assuming ddf_01 and ddf_16 are your original Dask DataFrames

# Convert all columns in both DataFrames to dtype 'object'
ddf_01_obj = ddf_01.astype(str)
ddf_16_obj = ddf_16.astype(str)

concatenated_ddf = dd.concat([ddf_01_obj, ddf_16_obj], axis=0)

# Compute the concatenated DataFrame to get the result
try:
    concatenated_result = concatenated_ddf.compute()
    print("Concatenated DataFrame computed successfully.")
    print(concatenated_result.head())
except Exception as e:
    print(f"Error computing the concatenated DataFrame: {e}")


In [ ]:
# Find common columns between the two DataFrames
common_columns = set(ddf_01.columns).intersection(set(ddf_16.columns))
print(len(common_columns))
print(common_columns)

In [ ]:
common_column = list(common_columns)

# Merge the datasets using an outer join on the common columns
merged_ddf = dd.merge(ddf_01, ddf_16, on=common_column, how='outer')

# Compute the merged DataFrame to get the result
merged_result = merged_ddf.compute()

# Show the first few rows of the merged DataFrame
print(merged_result.head())


#### Change some columns from object to category

In [ ]:
import pandas as pd
import numpy as np

def transform_data_types(df):
    for column in df.columns:
        # Check for NaN values in the column
        has_nan = df[column].isnull().any()

        if df[column].dtype == 'float64':
            # Convert to int16 if all values are integers and there are no NaN values
            if not has_nan and all(df[column].dropna().apply(float.is_integer)):
                df[column] = df[column].astype('int16')
            else:
                # Convert to float16 otherwise
                df[column] = df[column].astype('float16')

        elif df[column].dtype == 'object':
            # Convert string columns to category
            if all(df[column].apply(lambda x: isinstance(x, str) or pd.isnull(x))):
                df[column] = df[column].astype('category')

    return df

In [ ]:
TN_16_merged_data = transform_data_types(TN_16_merged_data)

print(TN_16_merged_data.info())
print(TN_16_merged_data.memory_usage(deep=True).sum())

#### change some columns from float64 to sparse[float16]

In [ ]:
summary_df_16 = pd.read_csv('TN_16_merged_data_summary.csv')
summary_df_16.info()

In [ ]:
#rename the column 'Unnamed: 0'
summary_df_16.rename(columns={'Unnamed: 0': 'header'}, inplace=True)

In [ ]:
# Filter based on Missing Percentage and Value Type
filtered_columns = summary_df_16[(summary_df_16['Missing Percentage'] > 85) & 
                             (summary_df_16['Value Type'].isin(['float64']))]

# Get the list of headers from the filtered DataFrame
tn_16_headers_with_missing_85 = filtered_columns['header'].tolist()

# Print the count of columns
print("Count of columns with missing Percentage > 85% and Value Type 'float64':", len(tn_16_headers_with_missing_85))

In [ ]:
error_columns = []  # List to store columns that raised errors

for col in tn_16_headers_with_missing_85:
    try:
        print("Before transformation of", col, ":", TN_16_merged_data[col].memory_usage(deep=True), "bytes")
        
        # Check data type of the column
        col_dtype = TN_16_merged_data[col].dtype
        print(col_dtype)
        if col_dtype == 'float16' or col_dtype == 'float64':
            # Convert numeric columns to default sparse type
            TN_16_merged_data[col] = pd.to_numeric(TN_16_merged_data[col], errors='coerce').astype('Sparse[float16]')
        # No need to replace the column in the original DataFrame
        # Just printing out the memory usage after transformation
        print("After transformation to Sparse of", col, ":", TN_16_merged_data[col].memory_usage(deep=True), "bytes")
    except ValueError as e:
        print(f"Error converting column {col}: {e}")
        error_columns.append(col)

    print()

# Print out columns that had errors
if error_columns:
    print("Columns with errors:", error_columns)

In [ ]:
#  
test_col = TN_16_merged_data['LowestGlucosemgdl'] 

print("original memory usage: ", test_col.memory_usage(deep=True))
test_col_32 = test_col.astype('Sparse[float16]')
print("memory usage as Int32: ", test_col_32.memory_usage(deep=True))

In [ ]:
print(TN_16_merged_data.info())
print(TN_16_merged_data.memory_usage(deep=True).sum())

In [ ]:
# Call the function and store the result
column_memory_tn_16= column_memory_usage(TN_16_merged_data)

# Display the result
print(column_memory_tn_16.info())

column_memory_tn_16.to_csv('TN_16_memory_usage_1.csv', index = False)

#### change MaskID into int32

##### There are some misfilled values in MaskID columns

In [ ]:
# Function to check if a value can be converted to a numeric type
def is_not_numeric(val):
    try:
        float(val)  # Try converting to float
        return False
    except ValueError:
        return True

# Apply the function to filter out non-numeric values
MaskID_non_numeric_values = TN_16_merged_data['MaskID'][TN_16_merged_data['MaskID'].apply(is_not_numeric)]
print(MaskID_non_numeric_values)

In [ ]:
# Function to check if a value is not an integer
def is_not_integer(val):
    try:
        float_val = float(val)  # Convert to float first
        int_val = int(float_val)
        return float_val != int_val  # Check if float value is not equal to its integer conversion
    except ValueError:
        return True  # Non-numeric values are also considered as not integer

# Apply the function to filter out non-integer values
MaskID_non_integer_values = TN_16_merged_data['MaskID'][TN_16_merged_data['MaskID'].apply(is_not_integer)]
print(MaskID_non_integer_values)

In [ ]:
print(TN_16_merged_data.info())
TN_16_merged_data = TN_16_merged_data.drop(MaskID_non_integer_values.index)
print(TN_16_merged_data.info())

In [ ]:
import pandas as pd

# Assuming TN_16_merged_data is your DataFrame and 'MaskID' is the column you're working with
test_col = TN_16_merged_data['MaskID']

print(test_col.memory_usage(deep=True))
print(test_col.head(30))

# Convert to numeric, coercing errors to NaN
test_col_numeric = pd.to_numeric(test_col, errors='coerce')
test_col_32 = test_col_numeric.astype('Int32')

print(test_col_32.memory_usage(deep=True))
print(test_col_32.head(30))
print()


In [ ]:
print(TN_16_merged_data.memory_usage(deep=True).sum())

test_col_numeric = pd.to_numeric(TN_16_merged_data['MaskID'], errors='coerce')
TN_16_merged_data['MaskID'] = test_col_numeric.astype('Int32')

print(TN_16_merged_data.memory_usage(deep=True).sum())

In [ ]:
TN_16_merged_data.info()

#### Change columns as object into category

In [ ]:
#  
test_col = TN_16_merged_data['AssessDateYear'] 

print("original memory usage as object: ", test_col.memory_usage(deep=True))
test_col_category = test_col.astype('category')
print("memory usage as category: ", test_col_category.memory_usage(deep=True))

test_col_numeric = pd.to_numeric(test_col, errors='coerce')
test_col_16 = test_col_numeric.astype('Int16')
print("memory usage as int16: ", test_col_16.memory_usage(deep=True))

In [ ]:
for column in TN_16_merged_data.columns:
    if TN_16_merged_data[column].dtype == 'object':
        print("original memory usage as object: ", TN_16_merged_data[column].memory_usage(deep=True))
        TN_16_merged_data[column] = TN_16_merged_data[column].astype('category')
        print("memory usage as category: ", TN_16_merged_data[column].memory_usage(deep=True))

In [ ]:
print(TN_16_merged_data.memory_usage(deep=True).sum())
TN_16_merged_data.info()

#### Turn Sparse[float64] into Sparse[float16]

In [ ]:
# Iterate over each column and convert if it's of type Sparse[float64]
for column in TN_16_merged_data.columns:
    if pd.api.types.is_sparse(TN_16_merged_data[column].dtype):
        if TN_16_merged_data[column].dtype.subtype == np.float64:
            TN_16_merged_data[column] = TN_16_merged_data[column].astype('Sparse[float16]')

print(TN_16_merged_data.info())
print()
print(TN_16_merged_data.memory_usage(deep=True).sum())

#### Turn SampleMaskID into int32

In [ ]:
# SampleMaskID has the biggest memory
# To turn it into interger, we need to check whether all values are starting with 'S', and the rest of values are integer.
column = TN_16_merged_data['SampleMaskID']

print("memory as category: ", column.memory_usage(deep=True))

test_col_obj = test_col.astype('object')
print("memory as object: ",test_col_obj.memory_usage(deep=True))

# Filter out missing values
non_missing_column = column[column.notna()]

# Check if values start with 'S'
starts_with_s = non_missing_column.str.startswith('S')

# Check if the rest part of the values are integers
rest_is_integer = non_missing_column.str.lstrip('S').apply(lambda x: x.isdigit())

# Combine the conditions
all_conditions_met = starts_with_s & rest_is_integer

# Check if all non-missing values in the column meet the conditions
all_values_meet_conditions = all_conditions_met.all()

print("Do all non-missing values start with 'S' and the rest part are integers?", all_values_meet_conditions)

##### No.1 change in column values: remove 'S' for 'SampleMaskID'

In [ ]:
# Strip 's' from the start of each string in the column
TN_16_merged_data['SampleMaskID'] = TN_16_merged_data['SampleMaskID'].str.lstrip('S')
print("original memory usage: ", TN_16_merged_data['SampleMaskID'].memory_usage(deep=True))

In [ ]:
# Convert the column to nullable integer type Int32
TN_16_merged_data['SampleMaskID'] = TN_16_merged_data['SampleMaskID'].astype('Int32')
print("memory as Int32: ", TN_16_merged_data['SampleMaskID'].memory_usage(deep=True))

#### turn float16 into category

In [ ]:
# Iterate over each column and convert if it's of type float16
for column in TN_16_merged_data.columns:
    if TN_16_merged_data[column].dtype == np.float16:
        print("memory used as float16: ", TN_16_merged_data[column].memory_usage(deep=True))
        #convert to float32 first
        TN_16_merged_data[column] = TN_16_merged_data[column].astype('float32')
        print("memory used as float32: ", TN_16_merged_data[column].memory_usage(deep=True))
        # Convert the column to 'category'
        TN_16_merged_data[column] = TN_16_merged_data[column].astype('category')
        print("memory used as category: ", TN_16_merged_data[column].memory_usage(deep=True))
        print()

In [ ]:
# Display DataFrame info and memory usage
print(TN_16_merged_data.info())
print()
print(TN_16_merged_data.memory_usage(deep=True).sum())

In [ ]:
shared_columns = TN_16_merged_data.columns.intersection(TN_01_merged_data.columns)
print(len(shared_columns))
print(shared_columns)

In [ ]:
# Check for inconsistencies in data types and only report those
inconsistencies = {}
for col in shared_columns:
    dtype_df1 = TN_01_merged_data[col].dtype
    dtype_df2 = TN_16_merged_data[col].dtype

    if dtype_df1 != dtype_df2:
        print(f"Column '{col}' has inconsistent data types: {dtype_df1} (DataFrame 1) != {dtype_df2} (DataFrame 2)")
    else:
        print(f"Column '{col}' has consistent data type: {dtype_df1}")

# Print only the columns with inconsistent data types
for col, dtypes in inconsistencies.items():
    print(f"Column '{col}' has inconsistent data types: {dtypes[0]} (DataFrame 1) vs {dtypes[1]} (DataFrame 2)")

In [ ]:
shared_columns = TN_16_merged_data.columns.intersection(TN_01_merged_data.columns)
print(len(shared_columns))
print(shared_columns)

In [ ]:
import pandas as pd

# Function to convert float16 columns to float32
def convert_float16_to_float32(df):
    float16_cols = df.select_dtypes(include=['float16']).columns
    for col in float16_cols:
        df[col] = df[col].astype('float32')
    return df

# Convert float16 columns to float32 for both DataFrames
TN_01_merged_data_float32 = convert_float16_to_float32(TN_01_merged_data)
TN_16_merged_data_float32 = convert_float16_to_float32(TN_16_merged_data)

# Find common headers
common_headers_01_16 = set.intersection(set(TN_01_merged_data_float32.columns), set(TN_16_merged_data_float32.columns))

# Merge the DataFrames on common headers using outer join
merged_data_01_16 = pd.merge(TN_01_merged_data_float32, TN_16_merged_data_float32, on=list(common_headers_01_16), how='outer', 
                            suffixes=('_dup_01', '_dup_16'))

print(merged_data_01_16.info())
merged_data_01_16.head()


In [ ]:
TN_01_merged_data.info()

In [ ]:
common_headers_01_16 = set.intersection(set(TN_01_merged_data.columns), set(TN_16_merged_data.columns))

# Merge the first two DataFrames on common headers using outer join
merged_data_01_16 = pd.merge(TN_01_merged_data, TN_16_merged_data, on=list(common_headers_01_16), how='outer', 
                            suffixes=('_duplication_x', '_duplication_y'))

print(merged_data_01_16.info())
merged_data_01_16.head()

## 1.3 TN_19

## 1.4 TN_20

#### test of merged dataset

In [ ]:
string = "D5OtherTestResultDD"  # change this to the string you're searching for
# Convert the search string to lower case (or upper case)
search_string_lower = string.lower()

# Find columns that contain the search string, case-insensitive
matching_columns = [col for col in transformed_merged_data.columns if search_string_lower in col.lower()]
print("Found ", len(matching_columns), "columns: ", matching_columns)

# Create a DataFrame with the filtered columns and drop rows with only missing values
filtered_df = transformed_merged_data[matching_columns].dropna(how='all')
print("DataFrame with filtered columns (rows with all missing values dropped):")
print()
print(filtered_df.info())
#print(filtered_df.head(30))

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(transformed_merged_data[col].value_counts().nlargest(30))
    print()

In [ ]:
summary_df = pd.read_csv('merged_first_part_summary.csv')

In [ ]:
merged_data_23.memory_usage(deep=True)

In [ ]:
merged_data_23['Date_of_Registration'] = pd.to_datetime(merged_data_23['Date_of_Registration'], format='%m/%d/%Y')

In [ ]:
merged_data_23.memory_usage(deep=True)

In [ ]:
merged_data_23.info()

In [ ]:
# Check unique value counts. 
unique_counts = merged_data_23.nunique()
unique_counts

# 1.4 Data query

## 1.4.1 check column names and their values containing a string

In [ ]:
# The following code is to load the first part of merged dataset
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import warnings
warnings.filterwarnings('ignore')

merged_first_part = pd.read_csv('TN_01_merged_data_1.csv')

Strings in columns that could be merged:
1. testName: 23
2. 
3. D5OtherTestResultDD, D5OtherTestResultMM, D5OtherTestResultYYYY: 3*23
4. 
5. 
6. 
7. 
8. Heightcm, heightin
9. Weightkg, Weightlbs
10.'TimeOfCollectionHH', 'TimeOfCollectionMM'
11.
12.
13. 

In [ ]:
merged_first_part.info()

In [ ]:
merged_first_part.info()

#### 1) Case insensitive

In [ ]:
string = "D6OtherTestResultUnit"  # change this to the string you're searching for
# Convert the search string to lower case (or upper case)
search_string_lower = string.lower()

# Find columns that contain the search string, case-insensitive
matching_columns = [col for col in merged_first_part.columns if search_string_lower in col.lower()]
print("Found ", len(matching_columns), "columns: ", matching_columns)

# Create a DataFrame with the filtered columns and drop rows with only missing values
filtered_df = merged_first_part[matching_columns].dropna(how='all')
print("DataFrame with filtered columns (rows with all missing values dropped):")
print()
print(filtered_df.info())
#print(filtered_df.head(30))

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_first_part[col].value_counts().nlargest(30))
    print()
    

#### 2) Case sensitive

In [ ]:
string = "D5OtherTestResult"  # Change this to the string you're searching for

# Find columns that contain the search string, case-sensitive
matching_columns = [col for col in merged_first_part.columns if string in col]
print("Found ", len(matching_columns), "columns: ", matching_columns)

# Create a DataFrame with the filtered columns and drop rows with only missing values
filtered_df = merged_first_part[matching_columns].dropna(how='all')
print("DataFrame with filtered columns (rows with all missing values dropped):")
print(filtered_df.info())
#print(filtered_df.head(30))

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_first_part[col].value_counts().nlargest(20))
    print()


## 1.4.2 check column names and their values containing two strings

Filtered columns:
1. 

In [ ]:
# Strings to search for: AND
string1 = ""  # First string
string2 = "heightin"  # Second string, change this to your second string

# Convert the search strings to lower case
search_string1_lower = string1.lower()
search_string2_lower = string2.lower()

# Find columns that contain both search strings, case-insensitive
matching_columns = [col for col in merged_first_part.columns 
                    if search_string1_lower in col.lower() and search_string2_lower in col.lower()]

print("Found ", len(matching_columns), "columns: ", matching_columns)

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_first_part[col].value_counts().nlargest(20))
    print()


In [ ]:
# Strings to search for: OR
string1 = "heightcm"  # First string
string2 = "heightin"  # Second string, change this to your second string

# Convert the search strings to lower case
search_string1_lower = string1.lower()
search_string2_lower = string2.lower()

# Find columns that contain both search strings, case-insensitive
matching_columns = [col for col in merged_first_part.columns 
                    if search_string1_lower in col.lower() or search_string2_lower in col.lower()]

print("Found ", len(matching_columns), "columns: ", matching_columns)

# Create a DataFrame with the filtered columns and drop rows with only missing values
filtered_df = merged_first_part[matching_columns].dropna(how='all')

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_first_part[col].value_counts().nlargest(20))
    print()

In [ ]:
filtered_df.describe()

## 1.4.3 check column headers in one shared CSV file

In [ ]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED

#file_path = ".././studies/TN_01/TrialNet_01_Data/TN01_ADVERSEEVENTS.CSV"
file_path = "/home/ec2-user/SageMaker/studies/TN_01/TrialNet_01_Data/TN01_nh08_diabonset_current.csv"

df1 = pd.read_csv(file_path)
df1.head()

In [ ]:
string = "MM"  # change this to the string you're searching for
# Convert the search string to lower case (or upper case)
search_string_lower = string.lower()

# Find columns that contain the search string, case-insensitive
matching_columns = [col for col in df1.columns if search_string_lower in col.lower()]
print("Found ", len(matching_columns), "columns: ", matching_columns)


matching_columns2 = ['MaskID', 'Visit_Dt'] + matching_columns
# Create a DataFrame with the filtered columns and drop rows with only missing values
filtered_df = df1[matching_columns2].dropna(how='all', subset=matching_columns)
print("DataFrame with filtered columns (rows with all missing values dropped):")
#print(filtered_df.info())
#print(filtered_df.head(30))

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(df1[col].value_counts().nlargest(20))
    print()


In [ ]:
# Display the selected rows
filtered_df.head(20)

In [ ]:
# Save the DataFrame to a CSV file
#filtered_df.to_csv('D5MeterReading_TN01_nh08_diabonset_current.csv')

In [ ]:
# Filter the dataframe based on the condition
filtered_rows = filtered_df[filtered_df['D5MeterReadingLoc1_1'] == 'Unknown']

# Display the first 20 rows of the filtered dataframe
filtered_rows.head(20)
